In [42]:
import pandas as pd
import numpy as np

In [43]:
fpath = './data/large.csv'

In [44]:
df = pd.read_csv(fpath)
# create a list that contains unique values in each column
unique_list = []
for i in range(0, len(df.columns)):
    unique_list.append(df.iloc[:, i].unique())
assert(set(unique_list[0]) == set(unique_list[3]))
state = np.sort(np.array(unique_list[0]))

In [45]:
nS = 500
nA = 9
N = np.zeros((nS, nA, nS))
Rho = np.zeros((nS, nA))
U = np.zeros((nS))

In [46]:
# load csv files one line at a time
with open(fpath, 'r') as f:
    for line in f:
        # skip header
        if line.startswith('s,a,r,sp'):
            continue
        s, a, r, sp = line.split(',')
        s_idx = np.where(state==int(s))[0].item()
        sp_idx = np.where(state==int(sp))[0].item()
        N[int(s_idx), int(a)-1, int(sp_idx)] += 1
        Rho[int(s_idx), int(a)-1] += int(r)

In [47]:
# Estimate of transition probabilities T(s'|s,a) ~ N(s'|s,a) / N(s,a)
T = N / N.sum(axis=2, keepdims=True)

# Estimate of reward function R(s,a) ~ Rho(s,a) / N(s,a)
R = Rho / N.sum(axis=2, keepdims=False)

# replace invalud values with 0
T[np.isnan(T)] = 0
R[np.isnan(R)] = 0

C:\Users\ralte\AppData\Local\Temp/ipykernel_9780/1272904333.py:2: RuntimeWarning: invalid value encountered in true_divide
  T = N / N.sum(axis=2, keepdims=True)
C:\Users\ralte\AppData\Local\Temp/ipykernel_9780/1272904333.py:5: RuntimeWarning: invalid value encountered in true_divide
  R = Rho / N.sum(axis=2, keepdims=False)


In [48]:
# Calculate action-value function Q(s,a) (vectorized)
gamma = 0.9
for i in range(100):
    Q = R + gamma * T.dot(U)
    U = Q.max(axis=1)

In [49]:
# output the policy based on Q
fpath = './data/large.policy'
policy = np.argmax(Q, axis=1)

In [51]:
with open(fpath, 'w') as f:
    for s in range(312020):
        if len(np.where(state==int(s+1))[0]) == 0:
            p = 0
        else:
            p = policy[np.where(state==int(s+1))[0].item()]
        f.write(str(p+1) + '\n')